In [1]:
!pip install -q datasets transformers tensorflow

In [2]:
import os, random, tensorflow as tf
from datasets import load_dataset
from tensorflow.keras.mixed_precision import set_global_policy
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

#enable training on gpu with quantized weights 
set_global_policy("mixed_float16")

2025-04-26 18:55:29.616380: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-26 18:55:29.616429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-26 18:55:29.617574: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-26 18:55:29.624700: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/dist-packages

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070 Laptop GPU, compute capability 8.6


2025-04-26 18:55:32.145825: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-26 18:55:32.155824: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-26 18:55:32.155859: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-26 18:55:32.156159: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
#download dataset
dataset = load_dataset("stanfordnlp/sst2")

#download gpt2 tokenizer and manually set pad token (gpt2 doesn't have one by default so we use the end of string token)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [4]:
#set max token sequence length and define the tokenizer function
MAX_LEN = 128
def tokenize_fn(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=MAX_LEN)

#adjust batch size higher or lower based on available VRAM. 32 has good performance on 8gb
BATCH_SIZE = 32

#create train and valudation datasets based on the ones included in sst2
train_ds = dataset["train"].map(tokenize_fn).to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=True,
    batch_size=BATCH_SIZE,
)
val_ds = dataset["validation"].map(tokenize_fn).to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=False,
    batch_size=BATCH_SIZE,
)


/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
2025-04-26 18:55:34.115778: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-26 18:55:34.115853: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/de

In [5]:
#load pre trained model
model = TFAutoModelForSequenceClassification.from_pretrained(
    "gpt2",            #load pretrained gpt2 weights
    num_labels=2       #set number of output classes (binary classification)
)


#adam optimizer with small learning rate for fine tuning
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)   

#using sparse categorical crossentropy for the loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  

#compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

#train model
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1      # train for 1 epoch because model is pre trained and any more causes overfitting
)

# save fine tuned model and tokenizer to file
save_dir = "./gpt2_sst2_finetuned"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

All PyTorch model weights were used when initializing TFGPT2ForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFGPT2ForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


   6/2105 [..............................] - ETA: 14:13 - loss: 2.6016 - accuracy: 0.5521

2025-04-26 18:56:02.865014: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f61f295d3e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-04-26 18:56:02.865076: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2025-04-26 18:56:02.874783: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-04-26 18:56:02.900814: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:467] Loaded cuDNN version 90000
I0000 00:00:1745693762.961943   24144 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2105/2105 [==============================] - 937s 434ms/step - loss: 0.2789 - accuracy: 0.8843 - val_loss: 0.2320 - val_accuracy: 0.9140


('./gpt2_sst2_finetuned/tokenizer_config.json',
 './gpt2_sst2_finetuned/special_tokens_map.json',
 './gpt2_sst2_finetuned/vocab.json',
 './gpt2_sst2_finetuned/merges.txt',
 './gpt2_sst2_finetuned/added_tokens.json',
 './gpt2_sst2_finetuned/tokenizer.json')

In [6]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

#load saved model and tokenizer
model = TFAutoModelForSequenceClassification.from_pretrained("./gpt2_sst2_finetuned")
tokenizer = AutoTokenizer.from_pretrained("./gpt2_sst2_finetuned")
tokenizer.pad_token = tokenizer.eos_token

#declare some test data
simple_reviews = [
    "I loved it.",
    "Amazing experience!",
    "Absolutely fantastic.",
    "Best film I've seen this year.",
    "Terrible and boring.",
    "Worst movie ever.",
    "Undeniably awful.",
    "Complete disaster."
]

ambiguous_reviews = [
    "It was okay, not great but not bad either.",
    "Some parts worked, others fell flat.",
    "I liked the idea more than the execution.",
    "Decent performances, weak story.",
    "Beautiful visuals, confusing plot.",
    "Good soundtrack, but I wouldn't watch it again.",
    "Enjoyable moments mixed with long boring scenes.",
    "Neither terrible nor amazing, just a movie.",
]

confusing_positive_reviews = [
    "I hated the theater, but the movie was actually pretty good.",
    "The plot was terrible, yet somehow I found it highly entertaining.",
    "It was a strange film, but the acting made it a fantastic experience.",
    "The first half was slow, but it ended really strong.",
    "Boring to start, but surprisingly emotional by the end.",
    "Terrible pacing, but unforgettable characters.",
    "Some parts were confusing, although I loved the visuals.",
    "I regret going, but I kind of liked the movie.",
]

confusing_negative_reviews = [
    "I loved the director's past work, but this movie was a total joke.",
    "The trailer looked great, but the movie itself was terrible.",
    "I enjoyed the first few minutes, but the rest was boring and predictable.",
    "It had a great cast, but they couldn't save the awful writing.",
    "Stunning visuals couldn't hide the fact that it was a complete mess.",
    "The premise was interesting, but the execution was painfully bad.",
    "I wanted to like it so badly, but it was just too slow and dull.",
    "There were some funny moments, but overall it was a huge waste of time.",
]

# function to run inference on a group of sentences
def run_inference(group_name, sentences):
    print(f"\n{group_name}\n")
    
    #tokenize the sentences
    encodings = tokenizer(
        sentences,
        padding=True,             #force pad all sentences to max length
        truncation=True,           #truncate if sentence is longer than max length
        return_tensors="tf",       #return tensorflow tensors
        max_length=128
    )

    #forward pass through the model
    logits = model(encodings).logits

    #apply softmax to convert logits to class probabilities
    probs = tf.nn.softmax(logits, axis=-1).numpy()

    #print predictions
    for sent, p in zip(sentences, probs):
        pred_label = "Positive" if p[1] > p[0] else "Negative"  #decide based on which probability is higher
        print(f"{sent:70s}  {pred_label}  (Pos: {p[1]:.1%} / Neg: {p[0]:.1%})")

# run inference on the sentences
run_inference("Simple Reviews", simple_reviews)
run_inference("Ambiguous Reviews", ambiguous_reviews)
run_inference("Confusing Positive Reviews", confusing_positive_reviews)
run_inference("Confusing Negative Reviews", confusing_negative_reviews)


All model checkpoint layers were used when initializing TFGPT2ForSequenceClassification.

All the layers of TFGPT2ForSequenceClassification were initialized from the model checkpoint at ./gpt2_sst2_finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2ForSequenceClassification for predictions without further training.



Simple Reviews

I loved it.                                                             Positive  (Pos: 99.7% / Neg: 0.3%)
Amazing experience!                                                     Positive  (Pos: 99.7% / Neg: 0.3%)
Absolutely fantastic.                                                   Positive  (Pos: 99.9% / Neg: 0.1%)
Best film I've seen this year.                                          Positive  (Pos: 99.8% / Neg: 0.2%)
Terrible and boring.                                                    Negative  (Pos: 0.1% / Neg: 99.9%)
Worst movie ever.                                                       Negative  (Pos: 0.3% / Neg: 99.7%)
Undeniably awful.                                                       Negative  (Pos: 0.2% / Neg: 99.8%)
Complete disaster.                                                      Negative  (Pos: 0.8% / Neg: 99.2%)

Ambiguous Reviews

It was okay, not great but not bad either.                              Positive  (Pos: 96.4% / Neg: 3.6%)
